<a href="https://colab.research.google.com/github/jxxngho/HG_MLDL/blob/main/05_2_Cross_validation_and_Grid_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 검증 세트

In [ ]:
# 테스트 세트를 사용하지 않으면 모델이 과대적합인지 과소적합인지 판단하기 힘들다.
# 테스트 세트를 사용하지 않고 이를 측정하는 간단한 방법은 훈련 세트를 또 나누는 것!
# 이를 검증 세트라고 한다.

# 훈련 세트에서 모델을 훈련하고 검증 세트로 모델을 평가
# 이런 식으로 테스트하고 싶은 매개변수를 바꿔가며 가장 좋은 모델을 고른다
# 그다음 이 매개변수를 이용해 훈련 세트와 검증 세트를 하벼 전체 훈련 데이터에서 모델을 다시 훈련
# 그리고 마지막에 테스트 세트에서 최종 점수를 평가한다!!!

In [2]:
# 데이터 불러오기
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [4]:
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [8]:
# 훈련 세트와 테스트 세트로 나누기

from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data,target, test_size=0.2, random_state=42)

print(train_input.shape, test_input.shape)

(5197, 3) (1300, 3)


In [6]:
# 이 다음 train_input, train_target을 다시 넣어 훈련 세트와 검증 세트를 만든다.

sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [7]:
# 단순히 train_test_split 함수를 두 번 적용해서 훈련 세트와 검증 세트로 나눈 것 뿐!
print(sub_input.shape, val_input.shape)

# 원래 5197개였던 훈련 세트가 4157개로 줄고, 검증세트는 1040개가 됨

(4157, 3) (1040, 3)


In [9]:
# 모델 학습 및 평가
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input,sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

# 훈련 세트에 과대적합 되어있다. 매개변수를 바꾸어 더 좋은 모델을 찾자!

0.9971133028626413
0.864423076923077


# 교차검증

In [10]:
# 검증 세트를 만드느라 훈련 세트가 줄었다 ㅠ_ㅠ
# 교차 검증은 검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복 -> 이 점수를 평균하여 최종 검증 점수를 얻는다.
# ex) k-폴드 교차 검증
# 사이킷런에서는 cross_validate() 교차 검증 함수 제공
# 평가할 모델 객체를 첫 번째 매개변수로 전달한 후 직접 검증 세트를  떼어 내지 않고 훈련 세트 전체를 cross_validate()함수에 전달

from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.02688432, 0.01064062, 0.01090527, 0.02055645, 0.02154183]), 'score_time': array([0.00188875, 0.0016799 , 0.00135994, 0.00178289, 0.00179482]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [ ]:
# 이 함수는 fit_time, score_time, test_score 키를 가진 딕셔너리를 반환
#처음 두 키는 각각 모델을 훈련하는 시간과 검증하는 시간을 의미
# cross_validate 기본값은 5

In [11]:
# 교차 검증의 최종 점수는 평균낼 수 있음

import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [12]:
# cross_validate()는 훈련 세트를 섞어 폴드를 나누지 않는다!
# train_test_split()함수로 전체 데이터를 섞은 후 훈련 세트를 준비했기 때문에 따로 섞을 필요가 없었다.
# 만약 섞을 필요가 있다면 분할기를 지정해줘야한다.

from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [13]:
# 만약 훈련 세트를 섞은 후 10-폴드 교차 검증 수행하려면?

splitter = StratifiedKFold(n_splits=10,shuffle=True,random_state=42)
scores = cross_validate(dt,train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [ ]:
# 결정 트리의 매개변수 값을 바꿔가며 가장 좋은 성능이 나오는 모델을 찾아보자~!
# 이 때 테스트 세트를 사용하지 않고 교차 검증을 통해서 좋은 모델을 고르면 된다.

# 하이퍼파라미터 튜닝

In [ ]:
# 하이퍼파라미터란 사용자 지정 파라미터이다.
# 자고 일어나서 마무리하자,,,